```
Copyright (c) Gradient Institute. All rights reserved.
Licensed under the Apache 2.0 License.
```


This notebook tests out this fix for estimating treatment effect in regularised linear models:

Hahn, P.R., Carvalho, C.M., Puelz, D., He, J., 2018. Regularization and Confounding in Linear Regression for Treatment Effect Estimation. Bayesian Anal. 13. https://doi.org/10.1214/16-BA1044


In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, BayesianRidge, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split

from twostageridge import TwoStageRidge

# Generate the data

In [2]:
N = 500
D_rand = 4  # concatenate this many random columns onto X
random_proj = True  # randomly project control variables
proj_D = 80

# Confounders - X
mu_x = np.zeros(4)
cov_x = np.array([
    [ 1.0,  0.5,  0.0,  0.3],
    [ 0.5,  1.0, -0.2,  0.0],
    [ 0.0, -0.2,  1.0, -0.7],
    [ 0.3,  0.0, -0.7,  1.0]
])


# Treatment - Z
gamma = np.array([2., 1., 0.0, -0.5])
sig_eps = 0.5


# Target - Y
alpha = .3
beta = np.array([1., -2., 1.5, 0.0])
sig_nu = 1.4

# Generation function
def generate_data():
    X = np.random.multivariate_normal(mean=mu_x, cov=cov_x, size=N)
    Z = X @ gamma + sig_eps * np.random.randn(N)
    Y = alpha * Z + X @ beta + sig_nu * np.random.randn(N)

    X = np.hstack((X, np.random.randn(N, D_rand)))
    if random_proj:
        D = X.shape[1]
        X = X @ np.random.standard_t(size=(D, proj_D), df=3)

    W = np.hstack((Z[:, np.newaxis], X))
    
    return W, Y

# Generate bias comparison results

In [3]:
S = 20
models = {
    "ols": LinearRegression(),
    "lasso": LassoCV(max_iter=20000),
    "ridge": RidgeCV(),
    "bayes": BayesianRidge(),
    "ts": TwoStageRidge(treatment_index=0, regulariser1=.1, regulariser2=.1)
}

scores = {k: [] for k in models.keys()}
mae = {k: [] for k in models.keys()}
rmse = {k: [] for k in models.keys()}

for s in range(S):
    W, Y = generate_data()
    W_train, W_test, Y_train, Y_test = train_test_split(W, Y, train_size=0.9)
    
    for name, model in models.items():
        model.fit(W_train, Y_train)
        alpha_hat = model.alpha_ if name == "ts" else model.coef_[0]
        err = alpha - alpha_hat
        abs_err = np.abs(err)
        sq_err = (err)**2
        
        scores[name].append(model.score(W_test, Y_test))
        mae[name].append(abs_err)
        rmse[name].append(sq_err)
        
    print('.', end='')
print()
        
        
scores = {k: np.mean(v) for k, v in scores.items()}
mae = {k: np.mean(v) for k, v in mae.items()}
rmse = {k: np.sqrt(np.mean(v)) for k, v in rmse.items()}


results = pd.DataFrame({
    "scores": scores,
    "mean absolute error": mae,
    "root mean square error": rmse
})

results

....................


,scores,mean absolute error,root mean square error
ols,0.749331,0.095803,0.127574
lasso,0.747343,0.147999,0.178170
ridge,0.749171,0.090936,0.116569
bayes,0.746883,0.210907,0.218878
ts,0.749265,0.092647,0.125752


In [4]:
models["ts"].score_stage1(W)

0.9675402526062132